# Data Cleaning Examples

In [1]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('AAPL.csv')
dataset.head(5), dataset.describe()

(         Date      Open      High       Low     Close  Adj Close     Volume
 0  1980-12-12  0.128348  0.128906  0.128348  0.128348   0.100453  469033600
 1  1980-12-15  0.122210  0.122210  0.121652  0.121652   0.095213  175884800
 2  1980-12-16  0.113281  0.113281  0.112723  0.112723   0.088224  105728000
 3  1980-12-17  0.115513  0.116071  0.115513  0.115513   0.090408   86441600
 4  1980-12-18  0.118862  0.119420  0.118862  0.118862   0.093029   73449600,
                Open          High           Low         Close     Adj Close  \
 count  10363.000000  10363.000000  10363.000000  10363.000000  10363.000000   
 mean      13.286884     13.429332     13.145575     13.292884     12.690122   
 std       28.487626     28.804560     28.182311     28.506974     28.230772   
 min        0.049665      0.049665      0.049107      0.049107      0.038434   
 25%        0.280803      0.286897      0.272500      0.280134      0.234073   
 50%        0.464286      0.473214      0.455357      0.4

## Filling nans with Local Mean Value

### Drop the unadjusted close column 

In [2]:
dataset = dataset.drop('Close', axis=1)
dataset.head(5)

,Date,Open,High,Low,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.100453,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.095213,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.088224,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.090408,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.093029,73449600


### Use Rolling Method to Replace Nans with Mean

In [3]:
def rolling(values, rolling_window_size=65, rolling_increment=1, rolling_start=0):
    values = np.asarray(values)
    for i in range(rolling_start, len(values)-rolling_window_size, rolling_increment):
        values_tmp = values[i:i+rolling_window_size]
        nan_indexes = np.where(values_tmp==np.nan)
        mean      = np.mean(values_tmp)
        for i in nan_indexes:
            values_tmp[i] = mean
    return values

columns_list = dataset.columns.to_list()[1:]
for i in columns_list:
    processed_values = rolling(dataset[i].values)
    dataset[i] = processed_values
    
dataset.head(5)

,Date,Open,High,Low,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.100453,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.095213,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.088224,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.090408,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.093029,73449600


### Use Rolling Method to Replace Local Maxima or Minima with Mean 
In this function, there are three data structure options: Pandas Series, Numpy Array, Python List. I used array because it is efficient in memory(RAM) usage and indexing time cost. Although a list is faster than an array in indexing, it takes up much more space in memory(RAM) when dealing with a large dataset.  
When setting the rolling_increment value, try to use 1 because it will avoid the for loop ends early due to the large increment and exhausted data length.

In [4]:
def rolling(values, rolling_window_size=65, rolling_increment=1, rolling_start=0):
    values = np.asarray(values)
    for i in range(rolling_start, len(values)-rolling_window_size, rolling_increment):
        values_tmp = values[i:i+rolling_window_size]
        left      = values_tmp.mean()-3*values_tmp.std()
        right     = values_tmp.mean()+3*values_tmp.std()
        max_index = np.where(values_tmp<=left)
        min_index = np.where(values_tmp>=right)
        mean      = np.mean(values_tmp)
        values_tmp[max_index] = mean
        values_tmp[min_index] = mean
    return values

columns_list = dataset.columns.to_list()[1:]#remove date column because there is no need to remove the large values in it
for i in columns_list:
    processed_values = rolling(dataset[i].values)
    dataset[i] = processed_values
    
dataset.head(5)

,Date,Open,High,Low,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.100453,42756430
1,1980-12-15,0.122210,0.122210,0.121652,0.095213,36108800
2,1980-12-16,0.113281,0.113281,0.112723,0.088224,30981612
3,1980-12-17,0.115513,0.116071,0.115513,0.090408,29579716
4,1980-12-18,0.118862,0.119420,0.118862,0.093029,73449600


## Resample the Dataset to Different Time Periods 

In [5]:
def resample(df, period):
    Xmin_df = pd.DataFrame()
    Xmin_df['open'] = df['Open'].resample(period).first()
    Xmin_df['high'] = df['High'].resample(period).max()
    Xmin_df['low'] = df['Low'].resample(period).min()
    Xmin_df['close'] = df['Adj Close'].resample(period).last()
    Xmin_df['volume'] = df['Volume'].resample(period).sum()
    try:
        Xmin_df['amount'] = df['Amount'].resample(period).sum()
    except:
        pass
    Xmin_df.dropna(inplace=True)
    return Xmin_df

dataset.Date = pd.to_datetime(dataset.Date)
dataset_monthly = resample(df= dataset.set_index('Date'), period= "1M")
dataset_monthly.head(5)

,open,high,low,close,volume
Date,,,,,
1980-12-31,0.128348,0.161272,0.112723,0.119234,541001875
1981-01-31,0.154018,0.155134,0.126116,0.098707,608988800
1981-02-28,0.119420,0.128906,0.106027,0.092592,321619200
1981-03-31,0.118862,0.120536,0.096540,0.085604,468905747
1981-04-30,0.108817,0.131138,0.108259,0.099143,500830328


## Rolling Standardization 
When applying standardization to a dataset, one likes to avoid the look-ahead bias. The assumption is that under an actual trading scenario, one could only receive the data of today and the days before today.  
While the standardization heavily relies on the mean and standard deviation values, the size of the sample and the dates the sample consists will affect the standardization results.  
If one applies standardization to an entire dataset all in once, then the realistic changing means and the std values under an actual trading scenario as new data keeps incoming is lost, and look ahead bias is generated because one is using the mean and std of the entire dataset rather than what data one could receive under an actual trading scenario.

In [6]:
rolling = 65
columns_list = dataset.columns.tolist()[1:]
for i in columns_list:
    dataset[i] = (dataset[i]-dataset[i].rolling(window=rolling,center=False).mean())/\
    dataset[i].rolling(window=rolling,center=False).std()
    
dataset = dataset.dropna()
dataset = dataset.reset_index(drop=True)
dataset.head(5)

,Date,Open,High,Low,Adj Close,Volume
0,1981-03-17,-1.189058,-1.139180,-1.163428,-1.163400,1.214530
1,1981-03-18,-0.761060,-0.713413,-0.737026,-0.737052,0.736794
2,1981-03-19,-0.784630,-0.804915,-0.794499,-0.794491,0.796855
3,1981-03-20,-0.753144,-0.707162,-0.729964,-0.729991,-0.853920
4,1981-03-23,-0.485329,-0.440203,-0.462679,-0.462673,-0.313549
